# dice_contract

In [ ]:
open rust
open rust.rust_operators

In [ ]:
//// test

open testing

## dice_contract

### util

In [ ]:
inl util () =
    !\($'"pub struct Util {} //"') : ()
    !\($'"impl Util { //"') : ()
    !\($'"    fn stream_u8_to_list( //"') : ()
    !\($'"        s: fable_library_rust::Native_::LrcPtr<dice_contract_lib::Dice::UH0>, //"') : ()
    !\($'"    ) -> dice_contract_lib::Dice::UH1 { //"') : ()
    !\($'"        match s.as_ref() { //"') : ()
    !\($'"            dice_contract_lib::Dice::UH0::UH0_0(n, f) => { //"') : ()
    !\($'"                dice_contract_lib::Dice::UH1::UH1_1( //"') : ()
    !\($'"                    *n, //"') : ()
    !\($'"                    Self::stream_u8_to_list(f()).into() //"') : ()
    !\($'"                ) //"') : ()
    !\($'"            } //"') : ()
    !\($'"            dice_contract_lib::Dice::UH0::UH0_1 => dice_contract_lib::Dice::UH1::UH1_0, //"') : ()
    !\($'"        } //"') : ()
    !\($'"    } //"') : ()
    !\($'"    fn list_u8_to_vec( //"') : ()
    !\($'"        s: fable_library_rust::Native_::LrcPtr<dice_contract_lib::Dice::UH1>, //"') : ()
    !\($'"    ) -> Vec<u8> { //"') : ()
    !\($'"        match s.as_ref() { //"') : ()
    !\($'"            dice_contract_lib::Dice::UH1::UH1_1(n, f) => { //"') : ()
    !\($'"                let mut v = Self::list_u8_to_vec(f.clone()); //"') : ()
    !\($'"                v.insert(0, *n); //"') : ()
    !\($'"                v //"') : ()
    !\($'"            } //"') : ()
    !\($'"            dice_contract_lib::Dice::UH1::UH1_0 => vec\![], //"') : ()
    !\($'"        } //"') : ()
    !\($'"    } //"') : ()
    !\($'"    fn vec_u8_to_list(v: Vec<u8>) -> dice_contract_lib::Dice::UH1 { //"') : ()
    !\($'"        v.iter() //"') : ()
    !\($'"            .rev() //"') : ()
    !\($'"            .fold(dice_contract_lib::Dice::UH1::UH1_0, |acc, x| { //"') : ()
    !\($'"                dice_contract_lib::Dice::UH1::UH1_1(*x, acc.into()) //"') : ()
    !\($'"            }) //"') : ()
    !\($'"    } //"') : ()
    !\($'"    fn vec_u8_to_stream(v: Vec<u8>) -> dice_contract_lib::Dice::UH0 { //"') : ()
    !\($'"        v.iter() //"') : ()
    !\($'"            .rev() //"') : ()
    !\($'"            .fold(dice_contract_lib::Dice::UH0::UH0_1, |acc, x| { //"') : ()
    !\($'"                dice_contract_lib::Dice::UH0::UH0_0( //"') : ()
    !\($'"                    *x, //"') : ()
    !\($'"                    fable_library_rust::Native_::Func0::new(move || acc.clone().into()), //"') : ()
    !\($'"                ) //"') : ()
    !\($'"            }) //"') : ()
    !\($'"    } //"') : ()
    !\($'"} //"') : ()

### state

In [ ]:
type state =
    {
        version : u32
        seeds : near.vector u8
    }

In [ ]:
//// test
///! rust -c

()

23:50:00   debug #1 near_workspaces.print_usd / { retry = 1; total_gas_burnt_usd = +0.000957; total_gas_burnt = 1432554003351 }
23:50:00   debug #2 near_workspaces.print_usd / outcome / { is_success = true; gas_burnt_usd = +0.000206; tokens_burnt_usd = +0.000206; gas_burnt = 308081859340; tokens_burnt = 30808185934000000000 }
23:50:00   debug #3 near_workspaces.print_usd / outcome / { is_success = true; gas_burnt_usd = +0.000602; tokens_burnt_usd = +0.000602; gas_burnt = 901289581511; tokens_burnt = 90128958151100000000 }
23:50:00   debug #4 near_workspaces.print_usd / outcome / { is_success = true; gas_burnt_usd = +0.000149; tokens_burnt_usd = +0.000000; gas_burnt = 223182562500; tokens_burnt = 0 }


In [ ]:
//// test
///! rust -c

trace Verbose (fun () => "") id

23:50:12   debug #1 near_workspaces.print_usd / { retry = 1; total_gas_burnt_usd = +0.001015; total_gas_burnt = 1519042951658 }
23:50:12   debug #2 near_workspaces.print_usd / outcome / { is_success = true; gas_burnt_usd = +0.000206; tokens_burnt_usd = +0.000206; gas_burnt = 308081859340; tokens_burnt = 30808185934000000000 }
23:50:12   debug #3 near_workspaces.print_usd / outcome / { is_success = true; gas_burnt_usd = +0.000660; tokens_burnt_usd = +0.000660; gas_burnt = 987778529818; tokens_burnt = 98777852981800000000 }
23:50:12   debug #4 near_workspaces.print_usd / outcome / { is_success = true; gas_burnt_usd = +0.000149; tokens_burnt_usd = +0.000000; gas_burnt = 223182562500; tokens_burnt = 0 }


### new

In [ ]:
inl new () : state =
    {
        version = 2
        seeds = "seeds" |> sm'.byte_slice |> near.new_vector
    }

In [ ]:
//// test
///! rust -c

inl state = new ()
trace Verbose (fun () => "dice_contract") fun () => { state = state |> sm'.format_debug }
trace Verbose (fun () => "") id

00:00:00 verbose #1 dice_contract / { state = (2, Vector { len: 0, prefix: [115, 101, 101, 100, 115] }) }
23:50:25   debug #1 near_workspaces.print_usd / { retry = 1; total_gas_burnt_usd = +0.001006; total_gas_burnt = 1505541412147 }
23:50:25   debug #2 near_workspaces.print_usd / outcome / { is_success = true; gas_burnt_usd = +0.000206; tokens_burnt_usd = +0.000206; gas_burnt = 308081859340; tokens_burnt = 30808185934000000000 }
23:50:25   debug #3 near_workspaces.print_usd / outcome / { is_success = true; gas_burnt_usd = +0.000800; tokens_burnt_usd = +0.000800; gas_burnt = 1197459552807; tokens_burnt = 119745955280700000000 }
23:50:25 warning #4 spiral_wasm.run / Error error / { retry = 1; error = "{ receipt_failures = []; receipt_outcomes_len = 1; retry = 1 }" }
00:00:00 verbose #1 dice_contract / { state = (2, Vector { len: 0, prefix: [115, 101, 101, 100, 115] }) }
23:50:33   debug #5 near_workspaces.print_usd / { retry = 2; total_gas_burnt_usd = +0.001006; total_gas_burnt = 150554

### max_seeds

In [ ]:
inl max_seeds () : unativeint =
    100i32 |> convert

### contribute_seed

In [ ]:
inl contribute_seed (seeds : rust.ref (rust.mut' (near.vector u8))) (seed : am'.vec u8) : () =
    !\($'"!seeds.extend(!seed); //"') : ()
    inl max_seeds = max_seeds ()

    inl seed_excess_len : i32 = !\($'"(!seeds.len() as i32) - !max_seeds as i32"')

    if seed_excess_len > 0 then
        inl seed_excess : am'.vec u8 =
            !\\(seed_excess_len, $'"!seeds.drain(0..$0 as u32).collect()"')
        trace Debug
            fun () => $'$"dice_contract.contribute_seed"'
            fun () => { seed_excess = seed_excess |> sm'.format_debug }

### generate_random_number

In [ ]:
inl generate_random_number
    (seeds : am'.vec u8) (key : sm'.std_string) (proof : sm'.std_string) (max : u64)
    : u64
    =

    inl seed = near.random_seed ()

    inl block_timestamp = near.block_timestamp ()
    inl block_height = near.block_height ()
    inl epoch_height = near.epoch_height ()
    inl account_balance = near.account_balance ()
    inl signer_account_id = near.signer_account_id ()

    inl entropy : am'.vec u8 =
        ;[
            seed
            seeds
            epoch_height |> am'.to_le_bytes |> am'.to_vec'
            block_height |> am'.to_le_bytes |> am'.to_vec'
            block_timestamp |> am'.to_le_bytes |> am'.to_vec'
            account_balance |> near.as_yoctonear |> am'.to_le_bytes |> am'.to_vec'
            signer_account_id |> am'.as_bytes |> am'.to_vec''
            proof |> sm'.into_bytes
            key |> sm'.into_bytes
        ]
        |> am'.to_vec
        |> rust.concat
    
    inl entropy_len = entropy |> am'.vec_len

    inl hash_u8 : am'.vec u8 = !\($'$"near_sdk::env::keccak512(&!entropy)"')

    !\($'"self.contribute_seed(!hash_u8.clone()); //"') : ()
    !\($'"let hash_stream = Util::vec_u8_to_stream(!hash_u8.clone()); //"') : ()
    !\($'"let rolls_list = Util::stream_u8_to_list(dice_contract_lib::Dice::rotate_numbers(6)( //"') : ()
    !\($'"    hash_stream.into(), //"') : ()
    !\($'")); //"') : ()
    inl rolls_list_log : am'.vec u8 = !\($'"Util::list_u8_to_vec(rolls_list.clone().into())"')

    trace Debug
        fun () => $'$"dice_contract.generate_random_number"'
        fun () =>
            {
                max
                key
                proof
                block_timestamp
                block_height
                epoch_height
                account_balance = account_balance |> sm'.format_debug
                signer_account_id = signer_account_id |> sm'.to_string
                seed = seed |> sm'.format_debug
                seeds = seeds |> sm'.format_debug
                entropy_len
                entropy = entropy |> sm'.format_debug
                hash_u8 = hash_u8 |> sm'.format_debug
                rolls_list_log = rolls_list_log |> sm'.format_debug
            }

    !\($'"let sequential_roll = //"') : ()
    !\($'"    dice_contract_lib::Dice::create_sequential_roller(rolls_list.into()); //"') : ()
    !\($'"let result = dice_contract_lib::Dice::roll_progressively //"') : ()
    !\($'"    (sequential_roll)(true)(max); //"') : ()

    trace Debug (fun () => "") (join id)

    !\($'"result as u64"')

### roll_within_bounds

In [ ]:
//// package=../dice

inl roll_within_bounds (max : u64) (rolls : am'.vec u8) : optionm'.option' u64 =

    // !\\(rolls, $'"let rolls = Util::vec_u8_to_list($0); //"') : ()

    inl rolls_list = rolls |> am'.from_vec |> fun (x : _ int _) => x |> am'.to_list' |> listm'.unbox
    inl result = rolls_list |> lib.dice.roll_within_bounds max
    // !\($'"let result = dice_contract_lib::Dice::roll_within_bounds(!max)(!rolls.into()); //"') : ()

    inl result = result |> optionm'.box

    trace Debug
        fun () => "dice_contract.roll_within_bounds"
        fun () => { max rolls = rolls |> sm'.format_debug; result }

    trace Debug (fun () => "") (join id)

    // !\($'"result"')

    result

In [ ]:
//// test
///! rust -c

inl rolls = ;[ 1; 5; 4; 4; 5 ] |> am'.to_vec

rolls
|> roll_within_bounds 2000
|> _assert_eq' (optionm'.some' 995)

00:00:00   debug #1 dice.accumulate_dice_rolls / { power = 4; acc = 0; roll = 1 }
00:00:00   debug #2 dice.accumulate_dice_rolls / { power = 3; acc = 0; roll = 5; value = 864 }
00:00:00   debug #3 dice.accumulate_dice_rolls / { power = 2; acc = 864; roll = 4; value = 108 }
00:00:00   debug #4 dice.accumulate_dice_rolls / { power = 1; acc = 972; roll = 4; value = 18 }
00:00:00   debug #5 dice.accumulate_dice_rolls / { power = 0; acc = 990; roll = 5; value = 4 }
00:00:00   debug #6 dice.accumulate_dice_rolls / { power = -1; acc = 994; result = 995 }
00:00:00   debug #7 dice_contract.roll_within_bounds / { max = 2000; rolls = [1, 5, 4, 4, 5]; result = Some(
    995,
) }
23:50:55   debug #1 near_workspaces.print_usd / { retry = 1; total_gas_burnt_usd = +0.002692; total_gas_burnt = 4030380386857 }
23:50:55   debug #2 near_workspaces.print_usd / outcome / { is_success = true; gas_burnt_usd = +0.000206; tokens_burnt_usd = +0.000206; gas_burnt = 308081859340; tokens_burnt = 3080818593400000000

### main

In [ ]:
///! _

inl main () =
    !\($'"} //"') : ()

    util ()

    !\($'"#[derive( //"') : ()
    !\($'"  near_sdk::PanicOnDefault, //"') : ()
    !\($'"  borsh::BorshDeserialize, //"') : ()
    !\($'"  borsh::BorshSerialize, //"') : ()
    !\($'")] //"') : ()

    !\($'"pub struct OldState { //"') : ()
    !\($'"    version: u32, //"') : ()
    !\($'"    seeds: near_sdk::store::vec::Vector<u8>, //"') : ()
    !\($'"} //"') : ()

    !\($'"#[near_sdk::near_bindgen] //"') : ()

    !\($'"#[derive( //"') : ()
    !\($'"  near_sdk::PanicOnDefault, //"') : ()
    !\($'"  borsh::BorshDeserialize, //"') : ()
    !\($'"  borsh::BorshSerialize, //"') : ()
    !\($'")] //"') : ()

    !\($'"pub struct State ( //"') : ()

    !\($'"/*"') : ()
    (null () : rust.type_emit state) |> ignore
    !\($'"*/ )"') : ()

    !\($'"impl From<OldState> for State { //"') : ()
    !\($'"    fn from(old_state: OldState) -> Self { //"') : ()
    !\($'"        Self((old_state.version + 1, old_state.seeds)) //"') : ()
    !\($'"    } //"') : ()
    !\($'"} //"') : ()

    inl new_ () =
        !\($'"#[init] //"') : ()
        !\($'"pub fn new() -> Self { // 1"') : ()

        (!\($'"true; /*"') : bool) |> ignore

        (null () : rust.type_emit ()) |> ignore

        (!\($'"true; */"') : bool) |> ignore

        inl result = new ()
        $'let _result = !result in _result |> (fun x -> Fable.Core.RustInterop.emitRustExpr x $"Self($0) // x") // 2' : ()

        !\($'"} // 2."') : ()

        !\($'"} // 1."') : ()

        2

    inl contribute_seed () =
        !\($'"pub fn contribute_seed(&mut self, seed: Vec<u8>) { //"') : ()
        inl seeds : rust.ref (rust.mut' (near.vector u8)) = !\($'$"&mut self.0.1"')
        inl seed : am'.vec u8 = !\($'$"seed"')

        seed |> contribute_seed seeds


        !\($'"} //"') : ()

        !\($'"} //"') : ()

        !\($'"} //"') : ()

        3

    inl contribute_seed_borsh () =
        !\($'"pub fn contribute_seed_borsh(&mut self, #[serializer(borsh)] seed: Vec<u8>) { //"') : ()
        !\($'"    self.contribute_seed(seed) //"') : ()
        !\($'"} //"') : ()

        1

    inl generate_random_number () =
        !\($'"pub fn generate_random_number(&mut self, key: String, proof: String, max: u64) -> u64 { //"') : ()

        inl key : sm'.std_string = !\($'$"key"')
        inl proof : sm'.std_string = !\($'$"proof"')
        inl max : u64 = !\($'$"max"')

        inl seeds : rust.ref (near.vector u8) = !\($'$"&self.0.1"')
        inl seeds : am'.vec u8 = !\($'$"!seeds.iter().map(|x| *x).collect::<Vec<u8>>()"')

        inl result = generate_random_number seeds key proof max

        !\($'"!result //"') : ()

        !\($'"} //"') : ()

        !\($'"} //"') : ()

        !\($'"} //"') : ()

        !\($'"} //"') : ()

        4

    inl roll_within_bounds () =
        !\($'"pub fn roll_within_bounds(&self, max: u64, rolls: Vec<u8>) -> Option<u64> { //"') : ()
        inl max : u64 = !\($'$"max"')
        inl rolls : am'.vec u8 = !\($'$"rolls"')
        inl result = roll_within_bounds max rolls

        !\\(result, $'"$0 //"') : ()

        !\($'"} //"') : ()

        !\($'"} //"') : ()

        2

    inl roll_within_bounds_borsh () =
        !\($'"#[result_serializer(borsh)] //"') : ()
        !\($'"pub fn roll_within_bounds_borsh( //"') : ()
        !\($'"    &self, //"') : ()
        !\($'"    #[serializer(borsh)] max: u64, //"') : ()
        !\($'"    #[serializer(borsh)] rolls: Vec<u8>, //"') : ()
        !\($'") -> Option<u64> { //"') : ()
        !\($'"    self.roll_within_bounds(max, rolls) //"') : ()
        !\($'"} //"') : ()

        1


    inl fns =
        [
            new_
            contribute_seed
            contribute_seed_borsh
            generate_random_number
            roll_within_bounds
            roll_within_bounds_borsh
        ]

    inl rec loop acc fns i =
        match fns with
        | [] => acc
        | x :: xs =>
            !\($'"#[near_sdk::near_bindgen] //"') : ()
            !\($'"impl State { //"') : ()
            inl n = x ()
            !\($'"} /* c"') : ()
            inl rec loop' i' =
                if i' <> 1 // <= n
                then (!\($'"true; */ // ???? / i: !i / i\': !i' / acc: !acc / n: !n"') : bool) |> ignore
                else
                    (!\($'"true; // ??? / i: !i / i\': !i' / acc: !acc / n: !n"') : bool) |> ignore
                    loop' (i' + 1)
            loop' 1u8
            loop (acc + n) xs (i + 1)
    inl n = loop 0u8 fns 1u8


    // !\($'"/* a"') : ()

    // !\($'"} // b"') : ()

    !\($'"fn _main() //"') : ()
    !\($'" //"') : ()

    inl rec loop' i' =
        if i' <= n
        then
            (!\($'"true; { (); // ?? / i\': !i' / n: !n"') : bool) |> ignore
            loop' (i' + 1)
        else
            (!\($'"true; { { (); // ? / i\': !i' / n: !n"') : bool) |> ignore
            // (!\($'"true; */ // ?? / i\': !i' / n: !n"') : bool) |> ignore
    loop' 1u8

inl main () =
    $'!main |> ignore' : ()